## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-30-18-51-33 +0000,nyt,Democrats Use Empty House to Shame G.O.P. Ahea...,https://www.nytimes.com/2025/09/30/us/politics...
1,2025-09-30-18-50-51 +0000,nyt,Judge Rules Trump Unlawfully Targeted Noncitiz...,https://www.nytimes.com/2025/09/30/us/politics...
2,2025-09-30-18-49-32 +0000,nyt,Beer Bust: Crash Sends Kegs and Cans Onto Cali...,https://www.nytimes.com/2025/09/30/us/beer-keg...
3,2025-09-30-18-46-42 +0000,nypost,MTA slams commuters with fare hike months afte...,https://nypost.com/2025/09/30/us-news/mta-slam...
4,2025-09-30-18-43-12 +0000,nyt,"Schools Superintendent in Iowa, Arrested by IC...",https://www.nytimes.com/2025/09/30/us/des-moin...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,60
6,shutdown,24
157,government,19
117,gaza,18
147,deal,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
240,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...,156
308,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...,149
112,2025-09-30-15-10-46 +0000,nypost,Trump gives Hamas ‘3 or 4 days’ to accept his ...,https://nypost.com/2025/09/30/world-news/trump...,128
285,2025-09-29-22-29-55 +0000,nypost,Ex-UK PM Tony Blair to help run Gaza under Tru...,https://nypost.com/2025/09/29/us-news/ex-uk-pm...,127
318,2025-09-29-20-46-24 +0000,bbc,Watch: Trump and Netanyahu outline peace plan ...,https://www.bbc.com/news/videos/c24r8zj1mnro?a...,126


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
240,156,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...
308,89,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...
62,54,2025-09-30-17-01-00 +0000,wsj,American Eagle Outfitters’ chief executive had...,https://www.wsj.com/business/retail/jay-schott...
29,46,2025-09-30-17-47-00 +0000,wsj,President Trump hailed the use of military for...,https://www.wsj.com/politics/national-security...
72,46,2025-09-30-16-30-00 +0000,wsj,Congress’s decision about what to do about exp...,https://www.wsj.com/politics/policy/government...
315,42,2025-09-29-20-53-00 +0000,wsj,The Trump administration said it is cracking d...,https://www.wsj.com/politics/national-security...
312,39,2025-09-29-21-00-00 +0000,latimes,Former state Senate leader Toni Atkins drops o...,https://www.latimes.com/california/story/2025-...
155,37,2025-09-30-12-50-51 +0000,nypost,Second Dallas ICE shooting victim dies from in...,https://nypost.com/2025/09/30/us-news/second-d...
321,35,2025-09-29-20-34-43 +0000,latimes,ChatGPT launches new parental controls for tee...,https://www.latimes.com/business/story/2025-09...
144,34,2025-09-30-13-33-28 +0000,nypost,Wild video shows drunk passenger being yanked ...,https://nypost.com/2025/09/30/world-news/drunk...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
